In [1]:
import tensorflow as tf
print("TensorFlow version is ", tf.__version__)

import math
import numpy as np
keras=tf.keras
from src import input_fn
import matplotlib.pyplot as plt
import sys
from tensorflow.keras.models import load_model
from src import config as cfg
import pandas as pd
import os
from tensorflow.keras.models import Model
from src.visualization_utils import plot_auroc_curve,plot_confusion_matrix_custom,plot_precision_recall_curve
import efficientnet.tfkeras as efn 

TensorFlow version is  2.0.0


In [2]:
# Model Parameters
batch_size = cfg.chexper_params['batch_size']
lr =  cfg.chexper_params['lr']
epoches = cfg.chexper_params['epoches']
image_size = cfg.chexper_params['image_size']

In [3]:
#input files
train = cfg.input_file['train']
test = cfg.input_file['test']

### <font color='red'> select labeling strategy here</font>

In [4]:
#labeling approach 
approach= 'u-Ignore'

In [5]:
#load path
model_dir = cfg.output_path[approach]['directory']
checkpoint_path = cfg.output_path[approach]['checkpoint_path']
model_name= cfg.output_path[approach]['model_name']

In [6]:
model_path= os.path.join('output',model_name)
print(model_path)

output/uIgnore.h5


In [7]:
#use previous checkpoint 
use_checkpoint=False

**finetune or transfer**

In [8]:
finetune = cfg.train_approach['finetune']

**Use ChexNet weights**

In [9]:
chexnet = cfg.train_approach['chexnet']

**use EfficientNet** 

In [10]:
efficientnet = cfg.train_approach['efficientnet']

## Train

### Load training data

In [11]:
# load training data
train_x,validation_x, train_y,validation_y=input_fn.process_training_data(file_path=train, approach=approach)
print(len(train_x))
print(len(train_y))
print(len(validation_x))
print(len(validation_y))

178731
178731
44683
44683


In [12]:
#Atelectasis,Cardiomegaly,Cosolidation,Edema,Pleural Effusion  
new_trainy=train_y[:,[8,2,6,5,10]]
new_validationy=validation_y[:,[8,2,6,5,10]]

### load tf.dataset format inputs

In [13]:
train_input= input_fn.input_fn_multi_output(True,train_x, new_trainy,batch_size)
eval_input= input_fn.input_fn_multi_output(False,validation_x,new_validationy,batch_size)

In [1]:
##custom accuracy for u-ignore to ignore y=-1 

def binary_accuracy(y_true, y_pred):
    t0 = tf.equal(y_true, 0)
    t1 = tf.equal(y_true, 1)
    p0 = tf.equal(tf.round(y_pred), 0)
    p1 = tf.equal(tf.round(y_pred), 1)
    everything = tf.reduce_sum(tf.cast(t0, tf.int32)) + tf.reduce_sum(tf.cast(t1, tf.int32))
    positives = tf.reduce_sum(tf.cast(tf.logical_and(t0, p0), tf.int32)) + tf.reduce_sum(tf.cast(tf.logical_and(p1, t1), tf.int32))
    return positives / everything

In [15]:
## custom weighted loss for u-ignore

def create_mask_weighted_binary_crossentropy(zero_weight, one_weight):

    def weighted_binary_crossentropy(y_true, y_pred):

        # Calculate the binary crossentropy
        b_ce=keras.backend.binary_crossentropy(tf.multiply(y_pred, tf.cast(tf.not_equal(y_true, -1), tf.float32)),
                                        tf.multiply(y_true, tf.cast(tf.not_equal(y_true, -1), tf.float32)))

        # Apply the weights
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce

        # Return the mean error
        return keras.backend.mean(weighted_b_ce)

    return weighted_binary_crossentropy

In [16]:
## custom weighted loss for u-one,u-zeros

def create_weighted_binary_crossentropy(zero_weight, one_weight):

    def weighted_binary_crossentropy(y_true, y_pred):

        # Calculate the binary crossentropy
        b_ce=keras.backend.binary_crossentropy(y_true, y_pred)

        # Apply the weights
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce

        # Return the mean error
        return keras.backend.mean(weighted_b_ce)

    return weighted_binary_crossentropy

In [17]:
#####citation!!####
#### got it form here https://gist.github.com/wassname/ce364fddfc8a025bfab4348cf5de852d ### 

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = keras.backend.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= keras.backend.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = keras.backend.clip(y_pred, keras.backend.epsilon(), 1 - keras.backend.epsilon())
        # calc
        loss = y_true * keras.backend.log(y_pred) * weights
        loss = -keras.backend.sum(loss, -1)
        return loss
    
    return loss

In [18]:
weight_list = input_fn.compute_weights(new_trainy)

In [19]:
weight_list

[{-1.0: 26987, 0.0: 124991, 1.0: 26753},
 {-1.0: 6468, 0.0: 150701, 1.0: 21562},
 {-1.0: 22247, 0.0: 144611, 1.0: 11873},
 {-1.0: 10352, 0.0: 126566, 1.0: 41813},
 {-1.0: 9391, 0.0: 100397, 1.0: 68943}]

In [20]:
if approach == 'u-MultiClass':
    loss1 = weighted_categorical_crossentropy([1, round(weight_list[1][0.0]/weight_list[1][1.0]), round(weight_list[1][0.0]/weight_list[1][2.0])]) 
    loss2 = weighted_categorical_crossentropy([1, round(weight_list[2][0.0]/weight_list[2][1.0]), round(weight_list[2][0.0]/weight_list[2][2.0])]) 
    loss3 = weighted_categorical_crossentropy([1, round(weight_list[3][0.0]/weight_list[3][1.0]), round(weight_list[3][0.0]/weight_list[3][2.0])]) 
    loss4 = weighted_categorical_crossentropy([1, round(weight_list[4][0.0]/weight_list[4][1.0]), round(weight_list[4][0.0]/weight_list[4][2.0])]) 
    loss5 = weighted_categorical_crossentropy([1, round(weight_list[5][0.0]/weight_list[5][1.0]), round(weight_list[5][0.0]/weight_list[5][2.0])]) 
elif approach == 'u-Ignore':
    loss1= create_mask_weighted_binary_crossentropy(1,round(weight_list[0][0.0]/weight_list[0][1.0]))
    loss2 = create_mask_weighted_binary_crossentropy(1,round(weight_list[1][0.0]/weight_list[1][1.0]))
    loss3 = create_mask_weighted_binary_crossentropy(1,round(weight_list[2][0.0]/weight_list[2][1.0]))
    loss4 = create_mask_weighted_binary_crossentropy(1,round(weight_list[3][0.0]/weight_list[3][1.0]))
    loss5 = create_mask_weighted_binary_crossentropy(1,round(weight_list[4][0.0]/weight_list[4][1.0]))
else:
    loss1= create_weighted_binary_crossentropy(1,round(weight_list[0][0.0]/weight_list[0][1.0]))
    loss2 = create_weighted_binary_crossentropy(1,round(weight_list[1][0.0]/weight_list[1][1.0]))
    loss3 = create_weighted_binary_crossentropy(1,round(weight_list[2][0.0]/weight_list[2][1.0]))
    loss4 = create_weighted_binary_crossentropy(1,round(weight_list[3][0.0]/weight_list[3][1.0]))
    loss5 = create_weighted_binary_crossentropy(1,round(weight_list[4][0.0]/weight_list[4][1.0]))

In [ ]:
col_names=['Atelectasis',
 'Cardiomegaly',
 'Cosolidation',
 'Edema',
 'PleuralEffusion']

### Construct the Model <br>
- Can be configured to use DenseNet-w ImageNet weights, EfficientNetB3-w ImageNet weights, DenseNet121-w Chexnet weights <br> 
- Also can be configured to do transfer learning or fine-tune

In [22]:
IMG_SHAPE=(image_size,image_size,3)

if chexnet==True:
    base_model=keras.applications.DenseNet121(weights=None, include_top=False, input_shape=IMG_SHAPE)
    base_model.load_weights("ChexNet_weights_notop.h5")
elif efficientnet==True:
    base_model=efn.EfficientNetB3(weights='imagenet', include_top=False,input_shape=IMG_SHAPE)
else:
    base_model=keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)


x = base_model.output
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256,activation='relu',name="fc") (x)
x = keras.layers.Dropout(0.5,name="dropout_final")(x)
if approach == 'u-MultiClass':
    num_outputs = 3
    activation = 'softmax'
else:
    num_outputs = 1
    activation= 'sigmoid'
### one output for each pathology    
out1 = keras.layers.Dense(num_outputs,activation=activation,name=col_names[0]) (x)
out2 = keras.layers.Dense(num_outputs,activation=activation,name=col_names[1]) (x)
out3 = keras.layers.Dense(num_outputs,activation=activation,name=col_names[2]) (x)
out4 = keras.layers.Dense(num_outputs,activation=activation,name=col_names[3]) (x)
out5 = keras.layers.Dense(num_outputs,activation=activation,name=col_names[4]) (x)


def get_layer_index(layer_name):
    for i, layer in enumerate(model.layers):
        if layer_name in layer.name:
            break;
    return i+1

adam=tf.keras.optimizers.Adam(lr=lr)
sgd =tf.keras.optimizers.SGD(lr = lr, momentum=0.8,nesterov=False) 

if finetune:
    model = Model(base_model.input,[out1, out2, out3, out4, out5])
    optimizer = sgd
else:
    base_model.trainable=False
    model = Model(base_model.input,[out1, out2, out3, out4, out5])
    optimizer = adam


if approach =='u-Ignore':
    accuracy = binary_accuracy
else:
    accuracy = keras.metrics.categorical_accuracy

model.compile(optimizer=optimizer,
             loss= {col_names[0]:loss1,
                   col_names[1]:loss2,
                   col_names[2]:loss3,
                   col_names[3]:loss4,
                   col_names[4]:loss5},
              loss_weights = {
                  col_names[0]:0.51,
                   col_names[1]:0.44,
                   col_names[2]:0.79,
                   col_names[3]:0.39,
                   col_names[4]:0.35},
             metrics=['accuracy',accuracy])

if finetune:
    for layer in model.layers[:get_layer_index('conv5_block13_2_conv')]:
    layer.trainable = False

In [ ]:
# model.summary()
        

In [24]:
if use_checkpoint==True:
    model.load_weights(os.path.join('output',checkpoint_path, 'cp.ckpt'))

In [25]:
steps_per_epoch = len(train_x) // batch_size
validation_steps = len(validation_x) // batch_size

In [26]:
##make directory for this approach 
model_direct = os.path.join('output',model_dir)
os.makedirs(model_direct,exist_ok=True)

### Create checkpoint

In [27]:
##make checkpoint path
checkpoint_path = os.path.join('output',model_dir,checkpoint_path)
os.makedirs(checkpoint_path,exist_ok=True)

In [28]:
##checkpoint file name
checkpoint_file = os.path.join(checkpoint_path,"cp.ckpt")

In [29]:
# Create checkpoint callback
checkpointer = tf.keras.callbacks.ModelCheckpoint(checkpoint_file,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_best_only=True,
                                                 mode='auto',
                                                 monitor='val_loss')
earlystopper=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,verbose=1)

**step decay** not used

In [30]:
# def step_decay(epoch):
#     init_lrate =0.001 
#     drop = 0.5 
#     epochs_drop = 2.0
#     lrate = init_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
#     return lrate

In [31]:
# lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [32]:
history = model.fit(train_input,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epoches,
                    validation_data=eval_input,
                    validation_steps=validation_steps,
                    callbacks=[checkpointer,earlystopper])

Train for 11170 steps, validate for 2792 steps
Epoch 1/20
  268/11170 [..............................] - ETA: 18:52 - loss: 27.4661 - Atelectasis_loss: 10.8622 - Cardiomegaly_loss: 12.6209 - Cosolidation_loss: 12.6581 - Edema_loss: 10.7372 - PleuralEffusion_loss: 6.2451 - Atelectasis_accuracy: 0.2397 - Atelectasis_binary_accuracy: 0.2829 - Cardiomegaly_accuracy: 0.3282 - Cardiomegaly_binary_accuracy: 0.3394 - Cosolidation_accuracy: 0.5864 - Cosolidation_binary_accuracy: 0.6700 - Edema_accuracy: 0.5562 - Edema_binary_accuracy: 0.5875 - PleuralEffusion_accuracy: 0.5468 - PleuralEffusion_binary_accuracy: 0.5769WARNING:tensorflow:Can save best model only with val_loss available, skipping.


KeyboardInterrupt: 

In [ ]:
model_path= os.path.join('output',model_name)

In [ ]:
model.save(model_path)

## Test 

In [ ]:
test_x,test_y=input_fn.process_testing_data(file_path=test)
test_input= input_fn.input_fn_multi_output(False,test_x, None,batch_size)

In [ ]:
model= load_model(model_path, compile=False)

In [ ]:
####needs to do if loading from history

# if approach =='u-Ignore':
#     model= load_model(model_path, custom_objects={'mask_binary_crossentropy':mask_binary_crossentropy})
# else:
#     model= load_model(model_path, compile=False)

In [ ]:
test_steps = len(test_x) // batch_size
result=model.predict(test_input,steps=test_steps)

In [ ]:
columns=["Pred_Atelectasis","Pred_Cardiomegaly","Pred_Consolidation","Pred_Edema","Pred_PleuralEffusion"]

### save test result csv

In [ ]:
resultdf= pd.DataFrame()
for i in range(len(result)):
    resultdf[columns[i]]=result[i].flatten()
resultdf=resultdf.join(test_y['Atelectasis'])
resultdf=resultdf.join(test_y['Cardiomegaly'])
resultdf=resultdf.join(test_y['Consolidation'])
resultdf=resultdf.join(test_y['Edema'])
resultdf=resultdf.join(test_y['Pleural Effusion'])
resultdf['image_path']=test_x[:len(resultdf)]
save_name = 'resources/'+'results_'+'weighted_'+'5_'+'Chex'+'_ImgAUg'+model_dir+'.csv'
resultdf.to_csv(save_name)
resultdf


In [ ]:
label_names=['Atelectasis',
'Cardiomegaly',
'Cosolidation',
'Edema',
'PleuralEffusion']

### plot AUROC curve

In [ ]:
plot_auroc_curve(resultdf, "uIgnore_imgAug_weighted_5_Chex_auroc.png",approach)

In [ ]:
y_pred=[]
for i in range(5):
    y_pred.append(( model.predict(test_input)[i] >0.4).astype(int))

result_hard_labels_df= pd.DataFrame()
for i in range(len(result)):
    result_hard_labels_df[columns[i]]=y_pred[i].flatten()
    
result_hard_labels_df=result_hard_labels_df.join(test_y['Atelectasis'])
result_hard_labels_df=result_hard_labels_df.join(test_y['Cardiomegaly'])
result_hard_labels_df=result_hard_labels_df.join(test_y['Consolidation'])
result_hard_labels_df=result_hard_labels_df.join(test_y['Edema'])
result_hard_labels_df=result_hard_labels_df.join(test_y['Pleural Effusion'])
#result_hard_labels_df

### Plot PRAUC curve

In [ ]:
#Get Confusion Matrices for all the labels

for i in range(len(y_pred)):    
    
    plot_confusion_matrix_custom(result_hard_labels_df.loc[:,result_hard_labels_df.columns[i+len(y_pred)]],result_hard_labels_df.loc[:,result_hard_labels_df.columns[i]],result_hard_labels_df.columns[i+len(y_pred)],approach)

In [ ]:
plot_precision_recall_curve(resultdf, "uIgnore_imgAUg_weighted_5_Chex_precision_recall.png",approach)

In [2]:
!jupyter nbconvert --to html 02_Train_Test.ipynb

[NbConvertApp] Converting notebook 02_Train_Test.ipynb to html
[NbConvertApp] Writing 369484 bytes to 02_Train_Test.html
